In [1]:
import os
import subprocess
SUBPROCESS_ENV = os.environ.copy()

In [2]:
import numpy as np

from PySDM_examples.Arabas_et_al_2015 import Settings, SpinUp
from PySDM_examples.utils.kinematic_2d import Simulation, Storage
from PySDM.exporters import VTKExporter
from PySDM_examples.utils import ProgBarController
import PySDM_examples
import glob
import platform
import pathlib

In [3]:
settings = Settings()
storage = Storage()
simulation = Simulation(settings, storage, SpinUp=SpinUp)
advectees_init_profiles = {
    "HDO vapour mixing ratio": np.full(shape=settings.grid[1], fill_value=1e-6),
    "H18O vapour mixing ratio": np.full(shape=settings.grid[1], fill_value=1e-6),
    "H17O vapour mixing ratio": np.full(shape=settings.grid[1], fill_value=1e-6),
}
simulation.reinit(additional_advectees_initial_profiles=advectees_init_profiles)

/Users/agnieszkazaba/PycharmProjects/PySDM/PySDM/backends/numba.py:48: UserWarning: Disabling Numba threading due to ARM64 CPU (atomics do not work yet)
  warnings.warn(


In [4]:
vtk_exporter = VTKExporter(path='.')    

simulation.run(ProgBarController("progress:"), vtk_exporter=vtk_exporter)
vtk_exporter.write_pvd()

FloatProgress(value=0.0, description='progress:', max=1.0)

In [5]:
pvanim = pathlib.Path(PySDM_examples.__file__).parent / "utils" / "pvanim.py"

product = pathlib.Path("./output/sd_products.pvd").absolute()
attributes = pathlib.Path("./output/sd_attributes.pvd").absolute()

try:
    result = subprocess.run(
        [
            "pvpython",
            "--force-offscreen-rendering",
            str(pvanim),
            str(product),
            str(attributes),
            str(pathlib.Path('./output').absolute()),
            "--animationname", "docs_intro_animation.ogv",
            "--animationframename", "last_animation_frame.pdf"
        ],
        check=platform.system() != "Windows",
        capture_output=True,
        text=True,
        env=SUBPROCESS_ENV,
    )
except subprocess.CalledProcessError as e:
    print(e.stderr)
    assert False